In [93]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [105]:
transforms_train = transforms.Compose([
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023,0.1994,0.2010))
])

transfomrs_test = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))
])

In [106]:
train_dev_dataset = torchvision.datasets.CIFAR10(root="./data", train = True, transform=transforms_train, download=True)
test_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, transform=transfomrs_test, download=True)

train_dataset, dev_dataset = torch.utils.data.random_split(train_dev_dataset, [45000,5000])

Files already downloaded and verified
Files already downloaded and verified


In [107]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [148]:
num_epochs = 25
batch_size = 64
drop_prob = 0.5
weight_decay_lambda = 1e-4
learning_rate = 0.001

In [158]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet,self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.fc_layer = nn.Sequential(
        nn.Linear(128*4*4, 1024),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256,10)
    )

  def forward(self, x):
    out = self.layer(x)
    out = out.view(out.size(0), -1)
    out = self.fc_layer(out)
    return out

In [159]:
def evalutaion(data_loader):
  correct = 0
  total = 0
  for images, labels in data_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  return correct / total

In [160]:

model = LeNet().to(device)

In [161]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True)
dev_loader = torch.utils.data.DataLoader(dataset=dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=False)

In [162]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.001, weight_decay=weight_decay_lambda)

In [163]:
loss_arr = []
max = 0
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
      loss_arr.append(loss)
      print('epoch [{}/{}], Step [{}/{}], Loss : {:.4f}'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

      with torch.no_grad():
        model.eval()
        acc = evalutaion(dev_loader)
        if max < acc:
          max = acc
          print("max dev accuracy : ", max)
          torch.save(model.state_dict(), "model.ckpt") # early stopping

epoch [1/25], Step [100/704], Loss : 1.8742
max dev accuracy :  0.2068
epoch [1/25], Step [200/704], Loss : 1.8626
max dev accuracy :  0.3662
epoch [1/25], Step [300/704], Loss : 1.4623
max dev accuracy :  0.4342
epoch [1/25], Step [400/704], Loss : 1.4591
max dev accuracy :  0.4742
epoch [1/25], Step [500/704], Loss : 1.5102
max dev accuracy :  0.5214
epoch [1/25], Step [600/704], Loss : 1.2090
max dev accuracy :  0.5456
epoch [1/25], Step [700/704], Loss : 1.1349
max dev accuracy :  0.5796
epoch [2/25], Step [100/704], Loss : 1.0536
max dev accuracy :  0.6072
epoch [2/25], Step [200/704], Loss : 1.0586
max dev accuracy :  0.6184
epoch [2/25], Step [300/704], Loss : 0.9426
epoch [2/25], Step [400/704], Loss : 0.9708
epoch [2/25], Step [500/704], Loss : 0.8366
max dev accuracy :  0.6546
epoch [2/25], Step [600/704], Loss : 1.0628
max dev accuracy :  0.6688
epoch [2/25], Step [700/704], Loss : 0.7811
epoch [3/25], Step [100/704], Loss : 0.6903
epoch [3/25], Step [200/704], Loss : 0.8372

In [164]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [165]:
print(correct / total)

0.7778
